In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [11]:
#fake text classification
df = pd.read_csv('/content/drive/My Drive/MY ML JOURNEY/Deep Learning Practice/Dataset/fake text classification/train.csv')

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
df=df.dropna()
X=df.drop('label',axis=1)
y=df['label']

In [14]:
X.shape,y.shape

((18285, 4), (18285,))

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional,Dense

In [16]:
voc_size=5000
txt = X.copy()
txt['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [17]:
txt.reset_index(inplace=True)

In [18]:
txt.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [21]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(txt)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', txt['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
onehot=[one_hot(words,voc_size)for words in corpus] 
sent_length=20
embedded=pad_sequences(onehot,padding='pre',maxlen=sent_length)

In [24]:
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded),y.shape

(18285, (18285,))

In [28]:
import numpy as np
X_final=np.array(embedded)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [31]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3074 - accuracy: 0.8570 - val_loss: 0.2095 - val_accuracy: 0.9145
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1435 - accuracy: 0.9475 - val_loss: 0.2186 - val_accuracy: 0.9157
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1028 - accuracy: 0.9641 - val_loss: 0.2495 - val_accuracy: 0.9125
Epoch 4/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0811 - accuracy: 0.9713 - val_loss: 0.2901 - val_accuracy: 0.9044
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0605 - accuracy: 0.9803 - val_loss: 0.2830 - val_accuracy: 0.9097
Epoch 6/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0493 - accuracy: 0.9829 - val_loss: 0.3990 - val_accuracy: 0.9056
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0331 - accuracy: 0.9905 - val_loss: 0.4446 - val_accuracy: 0.9021

In [32]:
y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(y_test,y_pred1)

array([[3055,  364],
       [ 203, 2413]])

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.9060480530240265

In [36]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.94      0.89      0.92      3419
           1       0.87      0.92      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.90      6035
weighted avg       0.91      0.91      0.91      6035



In [37]:
#without dropout
from tensorflow.keras.layers import Dropout
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
#model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)


Epoch 1/10
192/192 [==============================] - 5s 25ms/step - loss: 0.3036 - accuracy: 0.8584 - val_loss: 0.2158 - val_accuracy: 0.9125
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1408 - accuracy: 0.9456 - val_loss: 0.1982 - val_accuracy: 0.9171
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0945 - accuracy: 0.9662 - val_loss: 0.2618 - val_accuracy: 0.9145
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0660 - accuracy: 0.9782 - val_loss: 0.2776 - val_accuracy: 0.9039
Epoch 5/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0481 - accuracy: 0.9848 - val_loss: 0.3456 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0302 - accuracy: 0.9902 - val_loss: 0.4143 - val_accuracy: 0.9100
Epoch 7/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0201 - accuracy: 0.9944 - val_loss: 0.4099 - val_accuracy: 0.9090